# Quantity Discount (QD) Handler Module

Handles creation, activation, and deactivation of Quantity Discounts.

## Input Requirements
This module receives the **Module 3 output file** and processes Quantity Discounts.

**Required columns from Module 3:**
- `product_id` - Product ID
- `warehouse_id` - Warehouse ID
- `activate_qd` - Boolean (True = create/keep QD)
- `keep_qd_tiers` - List of tiers to keep, e.g., `['T1', 'T2']`
- `qd_tier_1_qty`, `qd_tier_1_disc_pct` - Tier 1 configuration
- `qd_tier_2_qty`, `qd_tier_2_disc_pct` - Tier 2 configuration
- `qd_tier_3_qty`, `qd_tier_3_disc_pct` - Tier 3 configuration

## Workflow
1. Load Module 3 output file
2. Deactivate ALL existing Quantity Discounts (clean slate)
3. Build QD configurations based on `keep_qd_tiers`
4. Create new QD where `activate_qd = True`

## Output
- Summary of actions taken
- Log file with details


In [ ]:
# =============================================================================
# IMPORTS & CONFIGURATION
# =============================================================================
import pandas as pd
import requests
from datetime import datetime, timedelta
import pytz
import glob
import os
import ast

# Cairo Timezone
CAIRO_TZ = pytz.timezone('Africa/Cairo')
CAIRO_NOW = datetime.now(CAIRO_TZ)
TODAY = CAIRO_NOW.date()

# =============================================================================
# API CONFIGURATION (TO BE FILLED WITH ACTUAL ENDPOINTS)
# =============================================================================
API_BASE_URL = "https://your-api-endpoint.com/api"
API_KEY = "your-api-key"
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

# Default QD settings
DEFAULT_QD_DURATION_HOURS = 12  # QD valid until next run

# =============================================================================
# INPUT FILE (Module 3 Output)
# =============================================================================
# Option 1: Specify exact file
# MODULE_3_OUTPUT = 'module_3_output_20260120_1500.xlsx'

# Option 2: Auto-find latest Module 3 output from today
def get_latest_module3_output():
    pattern = f'module_3_output_{TODAY.strftime("%Y%m%d")}_*.xlsx'
    files = glob.glob(pattern)
    if files:
        return max(files, key=os.path.getctime)
    return None

MODULE_3_OUTPUT = get_latest_module3_output()

print(f"Quantity Discount Handler")
print(f"{'='*50}")
print(f"Cairo Time: {CAIRO_NOW.strftime('%Y-%m-%d %H:%M')}")
print(f"Input File: {MODULE_3_OUTPUT}")


In [ ]:
# =============================================================================
# LOAD MODULE 3 OUTPUT
# =============================================================================
if MODULE_3_OUTPUT is None:
    raise FileNotFoundError("No Module 3 output file found for today!")

df = pd.read_excel(MODULE_3_OUTPUT)
print(f"Loaded {len(df)} records from Module 3 output")

# Filter to SKUs that need QD activation
df_to_activate = df[df['activate_qd'] == True].copy()

print(f"\nSKUs to activate QD: {len(df_to_activate)}")
print(f"Unique warehouses: {df_to_activate['warehouse_id'].nunique()}")

# Preview
if len(df_to_activate) > 0:
    print(f"\nPreview of SKUs to get QD:")
    cols_to_show = ['product_id', 'warehouse_id', 'sku', 'keep_qd_tiers', 'action_reason']
    cols_to_show = [c for c in cols_to_show if c in df_to_activate.columns]
    display(df_to_activate[cols_to_show].head(10))


In [ ]:
# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

def parse_keep_qd_tiers(value):
    """Parse keep_qd_tiers from string or list."""
    if pd.isna(value) or value is None:
        return []
    if isinstance(value, list):
        return value
    if isinstance(value, str):
        try:
            return ast.literal_eval(value)
        except:
            return []
    return []


def build_tiers_from_row(row, keep_tiers: list) -> list:
    """
    Build tier configuration from row data.
    
    Args:
        row: DataFrame row with qd_tier_X_qty and qd_tier_X_disc_pct columns
        keep_tiers: List of tiers to include, e.g., ['T1', 'T2']
        
    Returns:
        List of tier configs for create_qd()
    """
    tiers = []
    
    tier_map = {
        'T1': (1, 'qd_tier_1_qty', 'qd_tier_1_disc_pct'),
        'T2': (2, 'qd_tier_2_qty', 'qd_tier_2_disc_pct'),
        'T3': (3, 'qd_tier_3_qty', 'qd_tier_3_disc_pct')
    }
    
    for tier_name in keep_tiers:
        if tier_name in tier_map:
            tier_num, qty_col, disc_col = tier_map[tier_name]
            qty = row.get(qty_col, 0)
            disc = row.get(disc_col, 0)
            
            # Only include if both qty and discount are valid
            if qty and qty > 0 and disc and disc > 0:
                tiers.append({
                    "tier": tier_num,
                    "quantity": int(qty),
                    "discount_pct": float(disc)
                })
    
    return tiers

print("Helper functions defined ✓")


In [ ]:
# =============================================================================
# API FUNCTIONS (TO BE IMPLEMENTED WITH ACTUAL ENDPOINTS)
# =============================================================================

def deactivate_all_qd(warehouse_ids: list) -> dict:
    """
    Deactivate ALL active Quantity Discounts for given warehouses.
    
    Args:
        warehouse_ids: List of warehouse IDs to deactivate QD for
        
    Returns:
        dict with 'success', 'deactivated_count', 'errors'
    """
    # TODO: Replace with actual API call
    # Example:
    # response = requests.post(
    #     f"{API_BASE_URL}/quantity-discounts/bulk-deactivate",
    #     headers=HEADERS,
    #     json={"warehouse_ids": warehouse_ids}
    # )
    # return response.json()
    
    print(f"[STUB] Would deactivate all QD for warehouses: {warehouse_ids}")
    return {
        "success": True,
        "deactivated_count": 0,
        "errors": []
    }


def create_qd(
    product_id: int,
    warehouse_id: int,
    tiers: list,
    start_date: datetime = None,
    end_date: datetime = None,
    dynamic_tag_id: int = None
) -> dict:
    """
    Create a new Quantity Discount with specified tiers.
    
    Args:
        product_id: Product ID
        warehouse_id: Warehouse ID
        tiers: List of tier configs, e.g.:
            [
                {"tier": 1, "quantity": 5, "discount_pct": 3},
                {"tier": 2, "quantity": 10, "discount_pct": 5}
            ]
        start_date: When QD starts (default: now)
        end_date: When QD ends (default: now + DEFAULT_QD_DURATION_HOURS)
        dynamic_tag_id: Dynamic tag ID for warehouse targeting
        
    Returns:
        dict with 'success', 'qd_id', 'message'
    """
    if start_date is None:
        start_date = CAIRO_NOW
    if end_date is None:
        end_date = CAIRO_NOW + timedelta(hours=DEFAULT_QD_DURATION_HOURS)
    
    # TODO: Replace with actual API call
    # payload = {
    #     "product_id": product_id,
    #     "warehouse_id": warehouse_id,
    #     "start_at": start_date.isoformat(),
    #     "end_at": end_date.isoformat(),
    #     "dynamic_tag_id": dynamic_tag_id,
    #     "tiers": [
    #         {"quantity": t["quantity"], "discount_percentage": t["discount_pct"]}
    #         for t in tiers
    #     ]
    # }
    # response = requests.post(
    #     f"{API_BASE_URL}/quantity-discounts",
    #     headers=HEADERS,
    #     json=payload
    # )
    # return response.json()
    
    return {
        "success": True,
        "qd_id": None,
        "message": f"Created QD with {len(tiers)} tiers"
    }


def bulk_create_qd(qd_configs: list) -> dict:
    """
    Bulk create Quantity Discounts from list of configs.
    
    Args:
        qd_configs: List of dicts, each with:
            - product_id
            - warehouse_id
            - tiers (list of tier configs)
            
    Returns:
        dict with 'success', 'created_count', 'failed_count', 'errors'
    """
    created = 0
    failed = 0
    errors = []
    
    total = len(qd_configs)
    for idx, config in enumerate(qd_configs):
        result = create_qd(
            product_id=config['product_id'],
            warehouse_id=config['warehouse_id'],
            tiers=config['tiers']
        )
        if result['success']:
            created += 1
        else:
            failed += 1
            errors.append({
                'product_id': config['product_id'],
                'warehouse_id': config['warehouse_id'],
                'error': result.get('message', 'Unknown error')
            })
        
        # Progress update every 100 records
        if (idx + 1) % 100 == 0:
            print(f"  Progress: {idx + 1}/{total} processed...")
    
    return {
        "success": failed == 0,
        "created_count": created,
        "failed_count": failed,
        "errors": errors
    }

print("API functions defined ✓")


In [ ]:
# =============================================================================
# STEP 1: DEACTIVATE ALL EXISTING QUANTITY DISCOUNTS
# =============================================================================
print("STEP 1: Deactivating existing Quantity Discounts...")
print("="*50)

warehouse_ids = df['warehouse_id'].unique().tolist()
print(f"Warehouses to process: {warehouse_ids}")

deactivate_result = deactivate_all_qd(warehouse_ids)

print(f"\nDeactivation Result:")
print(f"  Success: {deactivate_result['success']}")
print(f"  Deactivated: {deactivate_result['deactivated_count']}")
if deactivate_result['errors']:
    print(f"  Errors: {len(deactivate_result['errors'])}")


In [ ]:
# =============================================================================
# STEP 2: BUILD QD CONFIGURATIONS
# =============================================================================
print("\nSTEP 2: Building QD configurations...")
print("="*50)

qd_configs = []
skipped = 0

if len(df_to_activate) == 0:
    print("No QD to create.")
else:
    for _, row in df_to_activate.iterrows():
        # Parse which tiers to keep
        keep_tiers = parse_keep_qd_tiers(row.get('keep_qd_tiers'))
        
        # If no specific tiers specified, include all available tiers
        if not keep_tiers:
            keep_tiers = ['T1', 'T2', 'T3']
        
        # Build tier configuration
        tiers = build_tiers_from_row(row, keep_tiers)
        
        if tiers:  # Only create if we have valid tiers
            qd_configs.append({
                'product_id': int(row['product_id']),
                'warehouse_id': int(row['warehouse_id']),
                'tiers': tiers,
                'sku': row.get('sku', 'N/A')
            })
        else:
            skipped += 1

print(f"Valid QD configs built: {len(qd_configs)}")
print(f"Skipped (no valid tiers): {skipped}")

# Preview
if qd_configs:
    print("\nSample QD configs:")
    for config in qd_configs[:5]:
        tier_str = ", ".join([f"T{t['tier']}:{t['quantity']}@{t['discount_pct']}%" for t in config['tiers']])
        print(f"  Product {config['product_id']}: [{tier_str}]")


In [ ]:
# =============================================================================
# STEP 3: CREATE NEW QUANTITY DISCOUNTS
# =============================================================================
print("\nSTEP 3: Creating new Quantity Discounts...")
print("="*50)

if len(qd_configs) == 0:
    print("No Quantity Discounts to create.")
    create_result = {"success": True, "created_count": 0, "failed_count": 0, "errors": []}
else:
    print(f"Creating {len(qd_configs)} Quantity Discounts...")
    create_result = bulk_create_qd(qd_configs)
    
    print(f"\nCreation Result:")
    print(f"  Success: {create_result['success']}")
    print(f"  Created: {create_result['created_count']}")
    print(f"  Failed: {create_result['failed_count']}")
    
    if create_result['errors']:
        print(f"\nErrors ({len(create_result['errors'])}):")
        for err in create_result['errors'][:10]:  # Show first 10
            print(f"  - Product {err['product_id']}, Warehouse {err['warehouse_id']}: {err['error']}")


In [ ]:
# =============================================================================
# SUMMARY
# =============================================================================
print("\n" + "="*60)
print("QUANTITY DISCOUNT HANDLER - SUMMARY")
print("="*60)
print(f"Timestamp: {CAIRO_NOW.strftime('%Y-%m-%d %H:%M')} Cairo Time")
print(f"Input File: {MODULE_3_OUTPUT}")
print(f"\nResults:")
print(f"  Total SKUs in input: {len(df)}")
print(f"  SKUs needing QD: {len(df_to_activate)}")
print(f"  Valid QD configs: {len(qd_configs)}")
print(f"  Skipped (no valid tiers): {skipped}")
print(f"  QD deactivated: {deactivate_result['deactivated_count']}")
print(f"  QD created: {create_result['created_count']}")
print(f"  QD failed: {create_result['failed_count']}")
print("="*60)

# Breakdown by number of tiers
if qd_configs:
    tier_counts = {}
    for config in qd_configs:
        n = len(config['tiers'])
        tier_counts[n] = tier_counts.get(n, 0) + 1
    print("\nQD by Number of Tiers:")
    for n, count in sorted(tier_counts.items()):
        print(f"  {n} tier(s): {count} QDs")
